In [1]:
%reset -f

# imports

In [1]:
from captum.attr import ShapleyValueSampling
from load_data import load_data

from segmentation import *
from utils import *
import torch
import os
from tqdm import tqdm
import timeit
import sys

# device for torch
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# hyper-parameters

In [2]:
batch_size = 50

dataset_names = { 'gunpoint'}    #{sys.argv[1]}
predictor_names = {'resNet'}    #{sys.argv[2]} {"randomForest", 'miniRocket', 'resNet'}
segmentation_names = [ "equal", "clasp" ,"greedygaussian", "infogain","nnsegment"]  # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} # {"clasp","greedygaussian", "equal", "infogain","nnsegment"} 
background_names = { "zero", "average","sampling"} #{"average", "zero", "sampling"}
normalization_names = {"default", "normalized"}

demo_mode = False
# demo
if demo_mode:
    dataset_names = {'UWAVE'}
    predictor_names = {"resNet"}
    segmentation_names = { "clasp" ,"greedygaussian", "infogain","nnsegment", "equal"} #,'clasp'}
    background_names ={ "sampling", "zero", "average"} #,'sampling'}
    normalization_names = {"default", "normalized"}


# instantiate dictionaries used in the pipeline

In [3]:
# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
    'torch' : ['resNet'],
    'scikit' : ['miniRocket','randomForest','QUANT']
}
segmentation_dict = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

results = dict.fromkeys(('y_test_true', 'label_mapping', "segments", 'y_test_pred', "attributions"))
for key in results.keys():
    results[key] = dict.fromkeys(dataset_names)
    
normalization_names = normalization_names | {"default"}


# train model

In [4]:
from models.predictor_utils import load_predictor, predict_proba
from models.train_models import *

for dataset_name in dataset_names:
    # init dataset and update the dict to be dumped
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
    results['y_test_true'][dataset_name] = y_test ; results['label_mapping'][dataset_name] = enc
    for k in ["attributions", "segments", "y_test_pred"] :
        results[k][dataset_name] =  dict.fromkeys(segmentation_names)
        
    # for debugging only
    if demo_mode:
        X_test = X_test[:2]
        y_test = y_test[:2]

    # TODO not to save if in demo mode!
    for predictor_name in predictor_names:

        if predictor_name=='resNet':
            clf = load_predictor("trained_models",predictor_name=predictor_name,dataset_name=dataset_name)
            preds = predict_proba(clf=clf,samples=X_test)
            #clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device, dir_name="trained_models")
        elif predictor_name=='miniRocket':
            #clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name, dir_name="trained_models")
            clf = load_predictor("trained_models",predictor_name=predictor_name,dataset_name=dataset_name)
            preds = predict_proba(clf=clf,samples=X_test)
        elif predictor_name=="randomForest":
            clf = load_predictor("trained_models",predictor_name=predictor_name,dataset_name=dataset_name)
            preds = predict_proba(clf=clf,samples=X_test)
            #clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name, dir_name="trained_models")
        elif predictor_name=="QUANT":
            #clf, preds = train_QUANT(X_train, y_train, X_test, y_test, dataset_name, dir_name="trained_models")
            clf = load_predictor("trained_models",predictor_name=predictor_name,dataset_name=dataset_name)
            preds = predict_proba(clf=clf,samples=X_test)
        else:
            raise ValueError("predictor not found")

        results['y_test_pred'][dataset_name][predictor_name] = preds


In [5]:
def initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results):
    
    init_segments = np.empty((X_test.shape[0], X_test.shape[1]), dtype=object) if X_test.shape[1] > 1 else ( np.empty(X_test.shape[0], dtype=object))
    results["segments"][dataset_name][segmentation_name] = init_segments.copy()
    results["attributions"][dataset_name][segmentation_name] = dict.fromkeys(predictor_names)
    for predictor_name in predictor_names:
        results["attributions"][dataset_name][segmentation_name][predictor_name] = dict.fromkeys(background_names)

In [6]:
def get_sample_info(segmentation_method, mask_list, ts_list, y_list, sample ,n_cps):
    
    # get current sample and label
    x, y = sample[0]  , torch.tensor((sample[1]))
    ts = torch.tensor(x)
    # get segment and its tensor representation
    current_segments = segmentation_method(x[0] , n_segments= n_cps)[:X_test.shape[1]]
    mask = get_feature_mask(current_segments, ts.shape[-1])
    
    # append any relevant information into the correct list
    mask_list.append(mask)
    ts_list.append(ts)
    y_list.append(y)
    
    return ts,y,mask, current_segments


In [7]:
from utils import sample_background

def get_background( background_name, X_train, n_background=50):

    # background data
    if background_name == "zero":
        background_dataset = torch.zeros((1,) + X_train.shape[1:])
    elif background_name == "sampling":
        background_dataset = sample_background(X_train, n_background)
    elif background_name == "average":
        background_dataset = torch.Tensor( X_train.mean(axis=0, keepdims=True) )

    return background_dataset


In [8]:
def get_attribution(explainer, ts, mask, background_dataset,y, sampling ): 
    
    if sampling:
        # get rid of first dimension as it's always 1
        # TODO try to flatten multiple singles "50 samples" into a 3D dataset and get the performances of that
        ts = ts[0] ;  mask= mask[0] ; y=y[0]

    if predictor_name in predictors['scikit']:
        
        tmp = explainer.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)

    elif predictor_name in predictors['torch']:
        # if use torch make sure everything is on selected device
        ts = ts.to(device); y = y.to(device) ; mask = mask.to(device); background_dataset = background_dataset.to(device)
        tmp = explainer.attribute(ts, target=y, feature_mask=mask, baselines=background_dataset)

    # in case of random forest 'un-flatten' result
    if predictor_name=="randomForest":
        tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

    # lastly store current explanation in the data structure; if sampling store the mean
    saliency_map = torch.mean(tmp, dim=0,  keepdim=True).cpu().numpy() if sampling else tmp.cpu().numpy()
    return saliency_map


In [9]:
def store_results(table, segmentation_name, normalization_names, current_results, start):
    
    # store "default" result
    n_results = current_results.shape[0]
    if 'default' in normalization_names:
        table['default'][start: (start+n_results) ] = current_results

    
    # and normalized result
    if "normalized" in normalization_names:
        weights = np.array(list(map(
            lambda segmentation: list(map(
                lambda channel_segemnts: lengths_to_weights(change_points_to_lengths(channel_segemnts, X_train.shape[-1])),
                segmentation)),
            results["segments"][dataset_name][segmentation_name][start: (start+n_results) ]  )))
        
        table['normalized'][start: (start+n_results) ]  = current_results * weights
    

# ideal n_cps to be found

In [10]:
# define number of cps to look for each dataset

ideal_n_cps = {
    "gunpoint" : 3,
    "UWAVE" : 7,
    "EOG" : 5 ,
    "MP8" : 8 ,
    "KeplerLightCurves" : 15,
}

# explain

In [11]:
from models.SHAP_dataloader import SHAP_dataloader
from torch.utils.data import DataLoader

starttime = timeit.default_timer()

with torch.no_grad():
    for dataset_name in dataset_names:
        
        for segmentation_name in segmentation_names:
            
            # initialize part of the dictionary and lists to be used
            initialize_result_dict(X_test,predictor_names,dataset_name,segmentation_name,results)
            segmentation_method = segmentation_dict[segmentation_name]
            
            ts_list = [] ; mask_list = [] ; y_list = []
            n_samples, n_chs, ts_length = X_test.shape

            # first run segmentation
            for i in range(n_samples) : 
                ts,y,mask, current_segment = get_sample_info( segmentation_method, mask_list, ts_list, y_list , sample=( X_test[i:i+1], y_test[i:i + 1]) ,n_cps = ideal_n_cps[dataset_name]  )
                results['segments'][dataset_name][segmentation_name][i] = current_segment

            for background_name in background_names:

                # get the background for the current data
                results["attributions"][dataset_name][segmentation_name][predictor_name][background_name] = dict.fromkeys(normalization_names)
                background_dataset = get_background( background_name, X_train)
                

                for predictor_name in predictor_names:
                    # get classifier and initialize attributions
                    init_attributions = np.zeros(X_test.shape, dtype=np.float32)
                    for normalization_name in normalization_names:
                        results['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name] = init_attributions.copy()

                    # instantiate SHAP explainer                    
                    SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] else ShapleyValueSampling(forward_classification)
                    
                    # prepare for batch computation
                    data_loader = DataLoader( SHAP_dataloader(ts_list,y_list,mask_list, background_dim=background_dataset.shape[0] ) ,
                        batch_size= 1 if background_name=='sampling' else batch_size)
                    
                    # computation loop
                    current_idx = 0
                    with tqdm(total=len(ts_list)) as pbar:
                        for (ts,y,mask) in data_loader:
        
                            current_results = get_attribution( SHAP, ts,mask,background_dataset,y, 
                                    sampling= (background_name=='sampling') )
                            
                            store_results(table=results['attributions'][dataset_name][segmentation_name][predictor_name][background_name], segmentation_name =segmentation_name, normalization_names=normalization_names,current_results=current_results, start=current_idx)
                            
                            # update counters
                            pbar.update(current_results.shape[0])  ; current_idx+=current_results.shape[0]
                            sys.stderr.flush() ; sys.stdout.flush()
                            
                    pbar.close()
                    
print("elapsed time", ( timeit.default_timer() -starttime ) )


 10%|█         | 15/150 [00:10<01:30,  1.49it/s]


KeyboardInterrupt: 

# ideal n_cps to be found

In [49]:
ideal_n_cps = {
    "gunpoint" : 3,
    "UWAVE" : 7 ,
    "EOG" : 5,
    "MP8" : 8,
    "KeplerLightCurves" : 15
}

In [50]:
# dump result to disk
if not demo_mode:
    file_name = "_".join( ("all_results",dataset_name,predictor_name) )
else:
	file_name = "_".join( ("all_results_DEMO_",dataset_name,predictor_name) )
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )